In [ ]:
import sys
import math

In [ ]:
def RegisteredNto1Adder(parameters, Print_To_File, Files_Location, Is_Top, modules):
    
    N = parameters["N"]
    
    Enable_Input_Regs = parameters["ENABLE_INPUT_REGISTERS"]

    #IN_WIDTH>=1
    IN_WIDTH = 10

    ModuleName="Registered{}to1Adder_".format(N)
    if Enable_Input_Regs<=0:
        ModuleName+="N"
    ModuleName+="IR"

    if Is_Top:
        Files_Location += ModuleName+"/"
        if not os.path.exists(Files_Location):
            os.makedirs(Files_Location)

    if Print_To_File<=0:
        of=sys.stdout
    else:
        of=open(Files_Location+ModuleName+".v", 'w+')

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for i in range(N):
        print("I{}, ".format(i), end='', file=of)
    print(file=of)
    print("output reg outReady = 0,", file=of)
    lgn=math.ceil(math.log(N,2))
    al=-1+lgn
    if al==0:
        print("output reg signed [IN_WIDTH:0] out,", file=of)
    else:
        print("output reg signed [IN_WIDTH+{}:0] out,".format(al), file=of)
    print("output ", end='', file=of)
    if Enable_Input_Regs>0:
        print("reg ", end='', file=of)
    print("earlyOutReady", end='', file=of)
    if Enable_Input_Regs>0:
        print(" = 0", file=of)
    else:
        print(file=of)
    print(");\n", file=of)

    if Enable_Input_Regs>0:
        print("reg signed [IN_WIDTH-1:0] ", end='', file=of)
        for i in range(N-1):
            print("I{}R, ".format(i), end='', file=of)
        print("I{}R;".format(N-1), file=of)

    print("always @(posedge clk) begin", file=of)
    print("\tif(reset) begin", file=of)
    if Enable_Input_Regs>0:
        print("\t\tearlyOutReady <= 0;", file=of)
    print("\t\toutReady <= 0;", file=of)
    print("\tend", file=of)
    print("\telse if(enable) begin", file=of)
    if Enable_Input_Regs>0:
        print("\t\tearlyOutReady <= inReady;", file=of)
        print("\t\toutReady <= earlyOutReady;", file=of)
        print("\t\tif(inReady) begin", file=of)
        for i in range(N):
            print("\t\t\tI{}R <= I{};".format(i, i), file=of)
        print("\t\tend", file=of)
    else:
        print("\t\toutReady <= inReady;", file=of)
    if Enable_Input_Regs>0:
        print("\t\tif(earlyOutReady) begin", file=of)
    else:
        print("\t\tif(inReady) begin", file=of)
    print("\t\t\tout <= ", end='', file=of)
    for i in range(N-1):
        print("I{}".format(i), end='', file=of)
        if Enable_Input_Regs>0:
            print("R+", end='', file=of)
        else:
            print("+", end='', file=of)
    print("I{}".format(N-1), end='', file=of)
    if Enable_Input_Regs>0:
        print("R;", file=of)
    else:
        print(";", file=of)
    print("\t\tend", file=of)
    print("\tend", file=of)
    print("end\n", file=of)

    if Enable_Input_Regs<=0:
        print("assign earlyOutReady = inReady;\n", file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()
        
    modules[ModuleName]="RegisteredNto1Adder"
    return ModuleName